In [ ]:
from typing import TypedDict, Literal

class PortfolioState(TypedDict):
    amount_usd: float
    total_usd: float
    target_currency: Literal['inr', 'eur']
    total: float

In [ ]:
def calc_total(state: PortfolioState) -> PortfolioState:
    state['total_usd'] = state['amount_usd'] * 1.08
    return state

def conv_inr(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd'] * 85.69
    return state

def conv_eur(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd'] * 0.85
    return state

def choose_conversion(state: PortfolioState) -> str:
    return state['target_currency']

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(PortfolioState)

builder.add_node("calc_total_node", calc_total)
builder.add_node("conv_inr_node", conv_inr)
builder.add_node("conv_eur_node", conv_eur)

builder.add_edge(START, "calc_total_node")
builder.add_conditional_edges(
    "calc_total_node",
    choose_conversion, {
    "inr": "conv_inr_node",
    "eur": "conv_eur_node"
})
builder.add_edge(["conv_inr_node", "conv_eur_node"], END)


graph = builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
amt = input("Enter amount in USD: ")

graph.invoke(
    {
        "amount_usd": float(amt),
        "target_currency": input("Enter target currency (inr/eur): ")
    }
)